بسم الله الرحمن الرحيم

**Première extraction**

Deuxième notebook.

Détails 1/2.

Le but ici est d'obtenir les détails de chaque livre : Titre arabe, nombre de volumes, édition....

## Première étape : aller chercher les urls de chacun des livres

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

r = requests.get('https://www.sifatusafwa.com/fr/fiqh-maliki/')
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', id = 'products')
#s = s.find_all('div', class_ = 'products product_list row grid')
#lines = s.find('h3', class_ = 's_title_block flex_child')
#Affichage du contenu de la page
#print(lines)

#Extraire les liens
urls = []
for line in s.findAll('a'):
    urls.append(line.get('href'))#line.text.split(\'n') est une liste de hcaine de caractères, donc [0] est 
    #seulement le str.
    #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n

Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.

In [2]:
#Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
urls = list(np.unique(urls))
#urls

On va supprimer les liens qui ne renvoient pas vers des livres : 
 - Ceux qui finissent pas '.asc' ou 'desc
 - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
 - les "javascript"

In [3]:
#Suppression des liens ne renvoyant pas vers des livres.
urls.remove('javascript:')
urls.remove('javascript:;')
urls.remove('https://www.sifatusafwa.com/fr/fiqh-maliki/')
to_supp = []
for link in range(len(urls)):
    if urls[link].endswith('asc') or urls[link].endswith('.desc') or urls[link].startswith('https://www.sifatusafwa.com/fr/fiqh-maliki/?'):
        to_supp.append(urls[link])
    else:
        to_supp.append('')
    

In [4]:
urls = pd.DataFrame(urls)
A = urls == pd.DataFrame(to_supp) 
urls = urls[A == False].dropna()

In [5]:
urls = urls.values.T.tolist()[0]

In [6]:
urls

['https://www.sifatusafwa.com/fr/fiqh-ahkam/charh-bidayah-al-mujtahid-wa-nihayah-al-muqtasid-abdullah-al-abadi.html',
 'https://www.sifatusafwa.com/fr/fiqh-ahkam/tabsirah-al-hukkam-fi-usul-al-aqdhiyyah-ibn-farhun-799h.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-jami-li-masa-il-al-mudawwanah-ibn-yunus-as-siqilli-451h.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-mou-amalat-fi-al-fiqh-al-maliki-sadiq-al-ghariani.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-muqaddimah-al-qurtubiyyah-bi-charh-cheikh-zarrouk-el-bernoussi.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/al-usul-fi-ma-rifah-al-usul-risalah-fi-usul-ad-din-ibn-al-arabi.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/charh-aqidah-malik-as-saghir-al-qayrawani-al-qadi-abdelwahhab.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/dalil-as-salik-ala-as-hal-al-massalik-houssain-as-soulaymani.html',
 'https://www.sifatusafwa.com/fr/fiqh-maliki/hachiyah-ad-dousouqi-ala-ach-charh-al-kabir-fiqh-mali

On va maintenant essayer de regrouper les données des liens dans un dataframe.

## Essai pour le premier lien

In [7]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))

In [8]:
#L_string

On essaye d'unifier pour enlever les deux "" et afficher en colonnes:

In [9]:
df = pd.DataFrame(L_string)
df = df.drop_duplicates() #des fois les volumes apparaissent deux fois
df.head() #Pour avoir un aperçu

,0,1,2,3
0,,Titre,شرح بداية المجتهد ونهاية المقتصد وبهامشه السبي...,
1,,Auteur,د. عبد الله العبادي,
2,,Volumes,4,
3,,Pages,2328,
4,,Édition,دار السلام,


In [10]:
df.columns = ['supp', 'categories', 'test', 'supp2']
df.drop(['supp', 'supp2'], axis = 1, inplace=True)

In [11]:
df.set_index('categories', inplace = True)
df.head()

,test
categories,
Titre,شرح بداية المجتهد ونهاية المقتصد وبهامشه السبي...
Auteur,د. عبد الله العبادي
Volumes,4
Pages,2328
Édition,دار السلام


الحمد لله, il faut maintenant transposer ce tableau et le faire tourner pour tous les sites.

## Tentative de concaténation de deux dataframes

On a pas réussi à concaténer les deux transposés, on va essayer de les concaténer et de les transposer après.

In [12]:
url = urls[1]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))
L_string = np.array(L_string)
df2 = pd.DataFrame((L_string))
df2 = df2.drop_duplicates()
df2 #Pour avoir un aperçu

,0,1,2,3
0,,Titre,تبصرة الحكام في أصول الأقضية ومناهج الأحكام,
1,,Auteur,ابن فرحون المالكي,
2,,Volumes,3,
3,,Pages,2013,
4,,Édition,دار القلم,
5,,Couverture,Rigide,
6,,Vérification et préparation,د. عثمان ضميرية,
8,,Format,17x24cm,
9,,Harakat,En Majorité ou en Totalité,


In [13]:
df2.columns = ['supp', 'categories', 'test', 'supp2']
df2.drop(['supp', 'supp2'], axis = 1, inplace=True)
df2.set_index('categories', inplace = True)
df2 #pour comparer avec df

,test
categories,
Titre,تبصرة الحكام في أصول الأقضية ومناهج الأحكام
Auteur,ابن فرحون المالكي
Volumes,3
Pages,2013
Édition,دار القلم
Couverture,Rigide
Vérification et préparation,د. عثمان ضميرية
Format,17x24cm
Harakat,En Majorité ou en Totalité


In [14]:
M = df.join(on='categories', other = df2, rsuffix='_df2', how = 'outer')

In [15]:
M.set_index(M['categories'], inplace = True)#Si il y a un tahqiq (authentification), une colonne supplémentaire apparaît,
#en faisant ca on dit que l'index sera la colonne "categories" avec le plus de champs remplis.
M.drop(['categories'], axis = 1, inplace = True)

In [16]:
#M

In [17]:
#M.T

Maintenant on peut essayer de faire une boucles sur toutes les pages. On pourra renommer les "categories" par le site où on pourra trouver ce livre.

## Essai de boucle sur tous les sites d'une page donnée

On fait juste le premier lien pour qu'on puisse fusionner sur un DataFrame, on le supprimera juste après inchAllah

In [18]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))
L_string = np.array(L_string)
df = pd.DataFrame((L_string))
df = df.drop_duplicates()#des fois les volumes apparaissent deux fois
df.columns = ['supp', 'categories', str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
df.drop(['supp', 'supp2'], axis = 1, inplace=True)
df.set_index('categories', inplace = True)

In [19]:
urls.remove(urls[0]) #On enlève le lien qu'on a déjà mis dans le dataframe de base

Changements dans la boucle:
 - df2 devient df_other
 - on ajoute le site en colonne

In [20]:
print(urls.index(urls[0]))

0


In [21]:
dfs = []
i = 0
for url in urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', class_='product-features')
    L_string = []
    for text in s.findAll(class_ ='data-sheet flex_container'):
        L_string.append(text.text.split('\n'))
    L_string = np.array(L_string)
    df_other = pd.DataFrame((L_string))
    df_other = df_other.drop_duplicates()#des fois les volumes apparaissent deux fois
    df_other.columns = ['supp', 'categories',str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
    df_other.drop(['supp', 'supp2'], axis = 1, inplace=True)
    df_other.set_index('categories', inplace = True)
    dfs.append(df_other)

In [22]:
M = pd.concat((d for d in dfs), axis = 1)

In [23]:
M.T

,Titre,Auteur,Volumes,Pages,Édition,Couverture,Vérification et préparation,Format,Harakat,Préface
https://www.sifatusafwa.com/fr/fiqh-ahkam/tabsirah-al-hukkam-fi-usul-al-aqdhiyyah-ibn-farhun-799h.html,تبصرة الحكام في أصول الأقضية ومناهج الأحكام,ابن فرحون المالكي,3,2013,دار القلم,Rigide,د. عثمان ضميرية,17x24cm,En Majorité ou en Totalité,NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/al-jami-li-masa-il-al-mudawwanah-ibn-yunus-as-siqilli-451h.html,الجامع لمسائل المدونة المسمى مصحف المذهب المالكي,ابن يونس الصقلي,10,5520,إبداع - دار ابن الجوزي القاهرة,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/al-mou-amalat-fi-al-fiqh-al-maliki-sadiq-al-ghariani.html,المعاملات في الفقه المالكي,الصادق بن عبد الرحمن الغرياني,NaN,333,دار ابن حزم,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/al-muqaddimah-al-qurtubiyyah-bi-charh-cheikh-zarrouk-el-bernoussi.html,شرح المقدمة القرطبية ليحيى القرطبي,الشيخ أحمد زروق البرنسي,NaN,334,دار ابن حزم,Rigide,د. أحسن زقور,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/al-usul-fi-ma-rifah-al-usul-risalah-fi-usul-ad-din-ibn-al-arabi.html,الوصول إلى معرفة الأصول,الإمام أبو بكر ابن العربي المالكي,NaN,172,دار الفتح,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/charh-aqidah-malik-as-saghir-al-qayrawani-al-qadi-abdelwahhab.html,شرح عقيدة مالك الصغير ابن أبي زيد القيرواني,القاضي عبد الوهاب بن نصر البغدادي,NaN,344,الرابطة المحمدية للعلماء في المغرب,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/dalil-as-salik-ala-as-hal-al-massalik-houssain-as-soulaymani.html,دليل السالك على أسهل المسالك,لحسن بن محمد سليماني,NaN,613,دار ابن حزم,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/hachiyah-ad-dousouqi-ala-ach-charh-al-kabir-fiqh-maliki.html,حاشية العلامة الدسوقي على الشرح الكبير للإمام ...,العلامة الدسوقي,15,NaN,دار ابن حزم,Rigide,أحمد بن علي الدمياطي,17x24cm,En Majorité ou en Totalité,NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/lawami-al-durar-fi-hatk-astar-al-mukhtasar-al-majlisi-ash-shinqiti.html,لوامع الدرر في هتك أستار المختصر,محمد المجلسي الشنقيطي,14,9032,إبداع - دار ابن الجوزي القاهرة,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN
https://www.sifatusafwa.com/fr/fiqh-maliki/le-statut-du-qabd-dans-la-prière-chez-l-imam-malik-al-mekki-ibn-azouz.html,هيئة الناسك في أن القبض في الصلاة هو مذهب الإم...,محمد المكي بن عزوز,NaN,176,دار الإمام مسلم,Rigide,NaN,17x24cm,Pas ou peu (ou que sur le Matn),NaN


الحمد لله Maintenant on peut essayer de généraliser à toutes les pages

## Généralisation à toutes les pages.

J'ai pris le même modèle de code que dans sifatusafwa-1.ipynb. Ici on va préparer la liste des urls.

In [24]:
##Aller chercher les urls des livres
urls = []
new_urls = []
to_supp =[]
NB_MAX_PAGES = 15
for i in range(1,NB_MAX_PAGES):
    i = str(i)
    r = requests.get('https://www.sifatusafwa.com/fr/fiqh-maliki/?page=' + i)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', id = 'products')
    #s = s.find_all('div', class_ = 'products product_list row grid')
    #lines = s.find('h3', class_ = 's_title_block flex_child')
    #Affichage du contenu de la page
    #print(lines)

    #Extraire les liens
    
    for line in s.findAll('a'):
        urls.append(line.get('href'))#line.text.split(\'n') est une liste de chaine de caractères, donc [0] est 
        #seulement le str.
        #print(line.text.split('\n'))#line.text enlève les balises et les choses propres à html/css. Le .split split sur les \n
    #Tous les liens sont en double/ triple... on va les faire apparaitre qu'une seule fois.
    urls = list(np.unique(urls))
    #urls
    # On va supprimer les liens qui ne renvoient pas vers des livres : 
    #  - Ceux qui finissent pas '.asc' ou 'desc
    #  - Les liens de page ("?page=2", "?page=3"...) et le lien 'fiqh-maliki/'
    #  - les "javascript"
    #Suppression des liens ne renvoyant pas vers des livres.

for link in urls:
    if (link.endswith('asc'))==False and \
        (link.endswith('.desc'))==False and \
        (link.startswith('https://www.sifatusafwa.com/fr/fiqh-maliki/?page=')) == False:
        new_urls.append(link)            
for i in range(3):
    try:
        new_urls.remove('javascript:')
        new_urls.remove('javascript:;')
        new_urls.remove('https://www.sifatusafwa.com/fr/fiqh-maliki/')
    except ValueError:
        continue
new_urls = pd.DataFrame(new_urls)
del urls
new_urls = new_urls.values.T.tolist()[0]
#on enlève tout ce qu'on a rajouté maintenant qu'on a pu faire la comparaison avec ce qu'il y avait
#a supprimer
urls = list(np.unique(new_urls))

A ce stade, `urls` devrait contenir les adresses de chacun des livres, pour toutes les pages de fiqh maliki.

In [25]:
#urls

Ici, on va construire les dataframes pour chaque page. 
Une page a plusieurs livres. On aura une seule liste qui contiendra toutes les livres de la page 1, ceux de la page2... et on concatènera tout inchAllah

In [26]:
url = urls[0]
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
s = soup.find('section', class_='product-features')
L_string = []
for text in s.findAll(class_ ='data-sheet flex_container'):
    L_string.append(text.text.split('\n'))
L_string = np.array(L_string)
df = pd.DataFrame((L_string))
df = df.drop_duplicates()#des fois les volumes apparaissent deux fois
df.columns = ['supp', 'categories', str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
df.drop(['supp', 'supp2'], axis = 1, inplace=True)
df.set_index('categories', inplace = True)
urls.remove(urls[0]) #On enlève le lien qu'on a déjà mis dans le dataframe de base


In [27]:
dfs = []
i = 0
for url in urls:
    i = i+1
    print('url numero', i, '/', len(urls), (i/len(urls))*100, "%")
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    s = soup.find('section', class_='product-features')
    L_string = []
    if s !=None:
        for text in s.findAll(class_ ='data-sheet flex_container'):
            L_string.append(text.text.split('\n'))
    else:
        print(url, "etape 1")
    L_string = np.array(L_string)
    df_other = pd.DataFrame((L_string))
    df_other = df_other.drop_duplicates()#des fois les volumes apparaissent deux fois
    if len(df_other.columns) == 4:
        df_other.columns = ['supp', 'categories',str(url), 'supp2']# a cette étape on remplace ce qu'il faut pour voir les sites en colonnes
        df_other.drop(['supp', 'supp2'], axis = 1, inplace=True)
        df_other.set_index('categories', inplace = True)
        dfs.append(df_other)
    else:
        print(url, 'etape 2')

url numero 1 / 139 0.7194244604316548 %
url numero 2 / 139 1.4388489208633095 %
url numero 3 / 139 2.158273381294964 %
url numero 4 / 139 2.877697841726619 %
url numero 5 / 139 3.597122302158273 %
url numero 6 / 139 4.316546762589928 %
url numero 7 / 139 5.0359712230215825 %
url numero 8 / 139 5.755395683453238 %
url numero 9 / 139 6.474820143884892 %
url numero 10 / 139 7.194244604316546 %
url numero 11 / 139 7.913669064748201 %
url numero 12 / 139 8.633093525179856 %
url numero 13 / 139 9.352517985611511 %
url numero 14 / 139 10.071942446043165 %
url numero 15 / 139 10.79136690647482 %
url numero 16 / 139 11.510791366906476 %
url numero 17 / 139 12.23021582733813 %
url numero 18 / 139 12.949640287769784 %
url numero 19 / 139 13.66906474820144 %
url numero 20 / 139 14.388489208633093 %
url numero 21 / 139 15.107913669064748 %
url numero 22 / 139 15.827338129496402 %
url numero 23 / 139 16.546762589928058 %
url numero 24 / 139 17.26618705035971 %
url numero 25 / 139 17.985611510791365 

In [28]:
M = pd.concat((d for d in dfs), axis = 1)
M = M.T

On peut rajouter le site en colonne, et changer l'index

In [29]:
M.head()

,Titre,Auteur,Pages,Édition,Couverture,Format,Harakat,Volumes,Vérification et préparation,Préface,Annotations
https://www.sifatusafwa.com/fr/aqida-croyance/fatawa-ulema-al-malikiyah-fi-taqrir-aqidah-ahl-as-sounnah.html,فتاوى علماء المالكية في تقرير عقيدة أهل السنة ...,د. فاضل بن نورالدين الإمام,871,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN
https://www.sifatusafwa.com/fr/aqida-croyance/jouhoud-al-malikiyah-fi-taqrir-tawhid-al-ibadah.html,جهود المالكية في تقرير توحيد العبادة,د. عبد الله بن فهد العرفج,487,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN
https://www.sifatusafwa.com/fr/biographies/malamih-min-hayat-al-imam-malik-ahmad-taha-rayan.html,ملامح من حياة الإمام مالك ، إمام دار الهجرة وص...,د. أحمد علي طه ريان,103,دار الفضيلة القاهرة,Souple,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN
https://www.sifatusafwa.com/fr/biographies/malik-ibn-anas-sirah-a-immah-al-madhahib-al-arba-ah.html,مالك بن أنس (سير أئمة المذاهب),عبد الغني الدقر,359,دار القلم,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN
https://www.sifatusafwa.com/fr/biographies/tartib-al-madarik-wa-taqrib-al-masalik-li-ma-rifat-a-lam-madhhab-malik.html,ترتيب المدارك وتقريب المسالك لمعرفة أعلام مذهب...,القاضي عياض,2051,مؤسسة الرسالة,Rigide,17x24cm,En Partie,4,NaN,NaN,NaN


In [30]:
M['site'] = M.index
M.set_index(np.arange(len(M)), inplace = True)
M.head()

,Titre,Auteur,Pages,Édition,Couverture,Format,Harakat,Volumes,Vérification et préparation,Préface,Annotations,site
0,فتاوى علماء المالكية في تقرير عقيدة أهل السنة ...,د. فاضل بن نورالدين الإمام,871,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...
1,جهود المالكية في تقرير توحيد العبادة,د. عبد الله بن فهد العرفج,487,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...
2,ملامح من حياة الإمام مالك ، إمام دار الهجرة وص...,د. أحمد علي طه ريان,103,دار الفضيلة القاهرة,Souple,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/biographies/mal...
3,مالك بن أنس (سير أئمة المذاهب),عبد الغني الدقر,359,دار القلم,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/biographies/mal...
4,ترتيب المدارك وتقريب المسالك لمعرفة أعلام مذهب...,القاضي عياض,2051,مؤسسة الرسالة,Rigide,17x24cm,En Partie,4,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/biographies/tar...


In [31]:
M.to_excel('livres.xlsx')

### Fin  de cette étape

On peut en bonus fusionner sur le fichier créé par sifatusafwa-1.ipynb pour avoir les titres en français. Je vois pas d'autre moyen que de fusionner sur le titre en arabe. L'index reposant sur l'ordre dans lequel les addresses, il peut différer.

## Essai d'ajout du nom des sites français

### Jointure sur titre arabe

In [32]:
M.columns

Index(['Titre', 'Auteur', 'Pages', 'Édition', 'Couverture', 'Format',
       'Harakat', 'Volumes', 'Vérification et préparation', 'Préface',
       'Annotations', 'site'],
      dtype='object')

In [33]:
df = pd.read_excel('fiqh_maliki.xlsx')
df = df.drop_duplicates()

FileNotFoundError: [Errno 2] No such file or directory: 'fiqh_maliki.xlsx'

In [ ]:
M['index'] = M.index

In [ ]:
df.shape

(136, 2)

In [ ]:
df.set_index('ar', inplace = True)

In [ ]:
M['ar'] = M['Titre']
M.set_index('ar', inplace = True)

In [ ]:
M_join = M.join(other =df, on = 'ar', rsuffix = '_df2', how = 'outer')

In [ ]:
print(df.shape, M.shape, M_join.shape)

(136, 1) (132, 13) (253, 15)


In [ ]:
M_join['fr'] = M_join['fr'].fillna(value = 'none')

On peut afficher les sites qui ont un `nan` dans la colonne `fr`

In [ ]:
M_join[M_join['fr'] == 'none']['site'].values

array(['https://www.sifatusafwa.com/fr/aqida-croyance/jouhoud-al-malikiyah-fi-taqrir-tawhid-al-ibadah.html',
       'https://www.sifatusafwa.com/fr/biographies/malamih-min-hayat-al-imam-malik-ahmad-taha-rayan.html',
       'https://www.sifatusafwa.com/fr/biographies/malik-ibn-anas-sirah-a-immah-al-madhahib-al-arba-ah.html',
       'https://www.sifatusafwa.com/fr/biographies/tartib-al-madarik-wa-taqrib-al-masalik-li-ma-rifat-a-lam-madhhab-malik.html',
       'https://www.sifatusafwa.com/fr/explications-de-livres-de-aqida/charh-aqidah-ibn-abi-zayd-al-qayrawani-cheikh-an-najmi.html',
       'https://www.sifatusafwa.com/fr/explications-de-livres-de-aqida/charh-usul-as-sunnah-lil-imam-ibn-abi-zamanin-zayd-al-madkhali.html',
       'https://www.sifatusafwa.com/fr/explications-de-recueils/at-tamhid-charh-al-muwatta-de-l-imam-ibn-abdil-barr.html#/21-edition-egyptienne',
       'https://www.sifatusafwa.com/fr/explications-de-recueils/at-tamhid-charh-al-muwatta-de-l-imam-ibn-abdil-barr.html#/23-

In [ ]:
M_join.isnull().sum()/len(M_join)

ar                             0.000000
Titre                          0.422925
Auteur                         0.438735
Pages                          0.648221
Édition                        0.596838
Couverture                     0.529644
Format                         0.525692
Harakat                        0.438735
Volumes                        0.845850
Vérification et préparation    0.877470
Préface                        0.960474
Annotations                    0.968379
site                           0.422925
index                          0.422925
fr                             0.000000
dtype: float64

40% des valeurs de site français n'ont pas été trouvés. On peut essayer de joindre sur l'index:

### Jointure sur l'index

In [ ]:
M.columns = ['Titre', 'Auteur', 'Pages', 'Édition', 'Couverture', 'Format',
       'Harakat', 'Volumes', 'Vérification et préparation', 'Préface',
       'Annotations', 'site', 'index_col']

In [ ]:
#On change l'index pour faire la jointure
M.set_index(M['index_col'], inplace = True)
M = M.drop(['index_col'], axis = 1)
df.set_index(np.arange(len(df)), inplace = True)

M_join = M.join(other =df, on = 'index_col', rsuffix = '_df2', how = 'outer')

#Aperçu des tailles après jointure
print(df.shape, M.shape, M_join.shape)

(136, 1) (132, 12) (136, 14)


On peut afficher les sites qui ont un `nan` dans la colonne `fr`

In [ ]:
M_join['fr'] = M_join['fr'].fillna(value = 'none')

In [ ]:
M_join.head(3)

,index_col,Titre,Auteur,Pages,Édition,Couverture,Format,Harakat,Volumes,Vérification et préparation,Préface,Annotations,site,fr
0.0,0,فتاوى علماء المالكية في تقرير عقيدة أهل السنة ...,د. فاضل بن نورالدين الإمام,871,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...,Majmou’ al-Amir (Moukhtasar al-Amir) - Fiqh Ma...
1.0,1,جهود المالكية في تقرير توحيد العبادة,د. عبد الله بن فهد العرفج,487,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...,Lawami' al-Durar fi Hatk Astar al-Mukhtasar - ...
2.0,2,ملامح من حياة الإمام مالك ، إمام دار الهجرة وص...,د. أحمد علي طه ريان,103,دار الفضيلة القاهرة,Souple,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/biographies/mal...,Al-Jami' li Masa-il al-Mudawwanah - Ibn Yunus ...


On voit que les titres francais correspondent pas aux titres arabes, comme on l'avait supposé, on va essayer d'utiliser l'url pour les extraire

### Titres français extraits avec l'url

In [ ]:
M.head(3)

,Titre,Auteur,Pages,Édition,Couverture,Format,Harakat,Volumes,Vérification et préparation,Préface,Annotations,site
index_col,,,,,,,,,,,,
0,فتاوى علماء المالكية في تقرير عقيدة أهل السنة ...,د. فاضل بن نورالدين الإمام,871,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...
1,جهود المالكية في تقرير توحيد العبادة,د. عبد الله بن فهد العرفج,487,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...
2,ملامح من حياة الإمام مالك ، إمام دار الهجرة وص...,د. أحمد علي طه ريان,103,دار الفضيلة القاهرة,Souple,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/biographies/mal...


In [ ]:
titresfr = M['site'].str.split('/')

In [ ]:
titresfr[:5]

index_col
0    [https:, , www.sifatusafwa.com, fr, aqida-croy...
1    [https:, , www.sifatusafwa.com, fr, aqida-croy...
2    [https:, , www.sifatusafwa.com, fr, biographie...
3    [https:, , www.sifatusafwa.com, fr, biographie...
4    [https:, , www.sifatusafwa.com, fr, biographie...
Name: site, dtype: object

In [ ]:
titresfr = titresfr.tolist()#pour que le DataFrame soit splitté par virgule.

In [ ]:
titresfr[:3]

[['https:',
  '',
  'www.sifatusafwa.com',
  'fr',
  'aqida-croyance',
  'fatawa-ulema-al-malikiyah-fi-taqrir-aqidah-ahl-as-sounnah.html'],
 ['https:',
  '',
  'www.sifatusafwa.com',
  'fr',
  'aqida-croyance',
  'jouhoud-al-malikiyah-fi-taqrir-tawhid-al-ibadah.html'],
 ['https:',
  '',
  'www.sifatusafwa.com',
  'fr',
  'biographies',
  'malamih-min-hayat-al-imam-malik-ahmad-taha-rayan.html']]

On remarque que le titre est en position 6...

In [ ]:
titresfr = pd.DataFrame(titresfr)

In [ ]:
#titresfr.head(3)

In [ ]:
titresfr.columns = ['col'+str(i) for i in range(1,8)] #On renomme les colonnes de titresfr

In [ ]:
titresfr.head(3)

,col1,col2,col3,col4,col5,col6,col7
0,https:,,www.sifatusafwa.com,fr,aqida-croyance,fatawa-ulema-al-malikiyah-fi-taqrir-aqidah-ahl...,None
1,https:,,www.sifatusafwa.com,fr,aqida-croyance,jouhoud-al-malikiyah-fi-taqrir-tawhid-al-ibada...,None
2,https:,,www.sifatusafwa.com,fr,biographies,malamih-min-hayat-al-imam-malik-ahmad-taha-ray...,None


In [ ]:
M['fr'] = titresfr['col6']

In [ ]:
M.head(2)

,Titre,Auteur,Pages,Édition,Couverture,Format,Harakat,Volumes,Vérification et préparation,Préface,Annotations,site,fr
index_col,,,,,,,,,,,,,
0,فتاوى علماء المالكية في تقرير عقيدة أهل السنة ...,د. فاضل بن نورالدين الإمام,871,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...,fatawa-ulema-al-malikiyah-fi-taqrir-aqidah-ahl...
1,جهود المالكية في تقرير توحيد العبادة,د. عبد الله بن فهد العرفج,487,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...,jouhoud-al-malikiyah-fi-taqrir-tawhid-al-ibada...


Renommer les colonnes et réordonner légèrement:

In [ ]:
M.rename(columns={'Titre':'Titre_ar', 'fr':'Titre_fr'}, inplace=True)

In [ ]:
titresfr = M['Titre_fr']

In [ ]:
M.drop(['Titre_fr'], axis = 1, inplace = True)

In [ ]:
M.insert(loc = 1, column = "Titre_fr", value = titresfr)

In [ ]:
M.head(2)

,Titre_ar,Titre_fr,Auteur,Pages,Édition,Couverture,Format,Harakat,Volumes,Vérification et préparation,Préface,Annotations,site
index_col,,,,,,,,,,,,,
0,فتاوى علماء المالكية في تقرير عقيدة أهل السنة ...,fatawa-ulema-al-malikiyah-fi-taqrir-aqidah-ahl...,د. فاضل بن نورالدين الإمام,871,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...
1,جهود المالكية في تقرير توحيد العبادة,jouhoud-al-malikiyah-fi-taqrir-tawhid-al-ibada...,د. عبد الله بن فهد العرفج,487,دار التوحيد للنشر,Rigide,17x24cm,Pas ou peu (ou que sur le Matn),NaN,NaN,NaN,NaN,https://www.sifatusafwa.com/fr/aqida-croyance/...


الحمد لله , on peut étendre aux autres domaines inchAllah

In [ ]:
M.to_csv('fiqh-maliki.csv')

Finalement on gardera les fichiers en csv.